In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.distributions import Categorical
import torch.optim as optim
import ast
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch.utils.data as Data
from sklearn.preprocessing import OneHotEncoder
import lightgbm as ltb
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
import scikitplot as skplt
from sklearn import tree
import csv
import pickle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
public_test = pd.read_csv("sampleSubmission.csv")
Y_test = np.genfromtxt("my_Y_test.csv", delimiter=',')
Y_train = np.genfromtxt("my_Y_train.csv", delimiter=',')
Y_valid = np.genfromtxt("my_Y_valid.csv", delimiter=',')

X = pd.read_csv("my_train_X.csv", header=None)
y = pd.read_csv("my_train_y.csv", header=None)
test = pd.read_csv("my_test.csv", header=None)

X_copy = X.copy()
y_copy = y.copy()
test_copy = test.copy()

X_arr = X_copy.to_numpy()
y_arr = y_copy.to_numpy()
test_arr = test_copy.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X_arr, y_arr, test_size=0.25, random_state=42)

X_train_t = torch.from_numpy(X_train.astype(np.float32))
y_train_t = torch.from_numpy(y_train.astype(np.float32))
X_test_t = torch.from_numpy(X_test.astype(np.float32))
y_test_t = torch.from_numpy(y_test.astype(np.float32))
test_X = torch.tensor(test_arr,dtype=torch.float32)

X_test_t = X_test_t.to(device)
y_test_t = y_test_t.to(device)
test_X = test_X.to(device)

In [3]:
regs = LinearRegression(fit_intercept = False).fit(Y_train, y_train)
weight = regs.coef_
weight

array([[ 1.23506103e+00,  7.56998310e+00,  2.44068501e-03,
        -7.64377281e+00, -1.68795368e-01]])

In [4]:
train_Y_pred = regs.predict(Y_train)
valid_Y_pred = regs.predict(Y_valid)
mean_squared_error(train_Y_pred, y_train) ** 0.5, mean_squared_error(valid_Y_pred, y_test) ** 0.5, mean_squared_error(train_Y_pred, y_train), mean_squared_error(valid_Y_pred, y_test)

(373.2962528844896, 374.1061004772205, 139350.09241760083, 139955.3744142722)

In [5]:
p = regs.predict(Y_test)
public_test["TRAVEL_TIME"] = p
public_test.to_csv("p.csv", index=None)

In [6]:
class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(MLP, self).__init__() #Refers to the fact that this is a subclass of nn.Module and is inheriting all methods
        self.model = torch.nn.Sequential( #an ordered container of modules
            nn.Linear(input_dim, hidden_dim),
           ## nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 64),
           ## nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(64,32),
           ## nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(32, 1)
        )   
    def forward(self, x): #You never have to call model.forward(x)
        """
        the forward function is where computatioin gets done
        """
        out = self.model(x)    
        return out

In [7]:
input_size = 5
hidden_size = 128
model_mlp = MLP(input_size, hidden_size)
model_mlp.to(device)
criterion = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model_mlp.parameters(), lr=0.00001)

In [8]:
def validate(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():  
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = (criterion(outputs, labels).item())*inputs.size(0)
            running_loss += loss
    return np.sqrt(running_loss / len(X_test))

In [9]:
Y_train_t = torch.from_numpy(Y_train.astype(np.float32))
y_train_t = torch.from_numpy(y_train.astype(np.float32))
Y_valid_t = torch.from_numpy(Y_valid.astype(np.float32))
y_test_t = torch.from_numpy(y_test.astype(np.float32))
test_y = torch.tensor(Y_test,dtype=torch.float32)

train_data = Data.TensorDataset(Y_train_t, y_train_t)
test_data = Data.TensorDataset(Y_valid_t, y_test_t)
train_loader = Data.DataLoader(dataset = train_data, batch_size = 30, 
                               shuffle = True, num_workers = 2)
test_loader = Data.DataLoader(dataset = test_data, batch_size = 30, 
                               shuffle = True, num_workers = 2)

In [15]:
rmses = []
mses = []
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    model_mlp.train()
    for i,(b_x, b_y) in tqdm(enumerate(train_loader)):
        inputs = b_x.to(device)
        labels = b_y.to(device)
        optimizer.zero_grad()
        prediction = model_mlp(inputs)
        loss = criterion(prediction,labels)
        running_loss +=loss.item()*inputs.size(0) 
        
        loss.backward()
        optimizer.step()
    rmse = np.sqrt(running_loss/len(X_train))
    validation_loss = validate(model_mlp, test_loader, criterion)
    rmses.append((rmse, validation_loss))
    mses.append((rmse ** 2, validation_loss ** 2))
    print(  'Epoch: {}/{}\t'.format(epoch+1, num_epochs),
            'Training Loss: {:.3f}\t'.format(rmse),
            'Validation Loss: {:.3f}\t'.format(validation_loss))

40932it [01:59, 342.67it/s]


Epoch: 1/10	 Training Loss: 394.131	 Validation Loss: 375.864	


40932it [01:53, 359.92it/s]


Epoch: 2/10	 Training Loss: 392.848	 Validation Loss: 375.287	


40932it [01:51, 368.40it/s]


Epoch: 3/10	 Training Loss: 392.524	 Validation Loss: 376.967	


13098it [00:38, 342.17it/s]


KeyboardInterrupt: 

In [ ]:
test_y = test_y.to(device)
prediction = model_mlp(test_y)
results = torch.squeeze(prediction, 1)
torch.save(model_mlp, "combined_mlp_10epoch.pth")

In [ ]:
result_list = []
for i in range(len(results)):
    result_list.append(results[i].item())
public_test["TRAVEL_TIME"] = result_list
public_test.to_csv("my_pred_mlps_combined.csv", index=None)
public_test.head()

In [ ]:
rmses, mses